## Data cleaning

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/yunlei/Desktop/MGMT 478/Combined dataset_nonsort.csv')

In [3]:
data.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,SNOW,TAVG,TMAX,TMIN
0,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-01,8.5,1.01,NaN,23.5,29.5,17.5
1,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-02,7.6,0.61,NaN,26,32.8,19.1
2,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-03,7.2,3.22,NaN,44.8,55.1,34.6
3,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-04,8.1,2.49,NaN,58.1,70.4,45.8
4,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-05,6.7,5.55,NaN,64.7,75.2,54.2


In [4]:
from sklearn.impute import SimpleImputer

In [5]:
# Convert DATE column to datetime format
data['DATE'] = pd.to_datetime(data['DATE'])

In [6]:
# Extract year and month from DATE as new features
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.month

In [7]:
# Drop the 'SNOW' column
data_cleaned = data.drop(['SNOW'], axis=1)

In [8]:
# Convert non-numeric to numeric
for column in ['LATITUDE','LONGITUDE','ELEVATION','AWND', 'TAVG', 'TMAX', 'TMIN']:
    data_cleaned[column] = pd.to_numeric(data_cleaned[column], errors='coerce')

In [9]:
# Imputer missing data as median of the column
imputer = SimpleImputer(strategy='median')
data_cleaned[['AWND', 'TAVG', 'TMAX', 'TMIN']] = imputer.fit_transform(data_cleaned[['AWND', 'TAVG', 'TMAX', 'TMIN']])

In [10]:
data_cleaned.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TAVG,TMAX,TMIN,YEAR,MONTH
0,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-01-01,8.5,1.01,23.5,29.5,17.5,2010,1
1,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-02-01,7.6,0.61,26.0,32.8,19.1,2010,2
2,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-03-01,7.2,3.22,44.8,55.1,34.6,2010,3
3,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-04-01,8.1,2.49,58.1,70.4,45.8,2010,4
4,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-05-01,6.7,5.55,64.7,75.2,54.2,2010,5


In [11]:
data_cleaned.isnull().sum()

STATION      0
NAME         0
LATITUDE     0
LONGITUDE    0
ELEVATION    0
DATE         0
AWND         0
PRCP         8
TAVG         0
TMAX         0
TMIN         0
YEAR         0
MONTH        0
dtype: int64

## Using KNN for finding the nearest stations

In [40]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    '''
    Define the Haversack formula function to calculate distance by latitude and longitude
    '''

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Mean radius of the Earth in kilometers
    return c * r

In [41]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

stations = data_cleaned[['STATION', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
coordinates = stations[['LATITUDE', 'LONGITUDE']]
neighbors_model = NearestNeighbors(n_neighbors=5)
neighbors_model.fit(coordinates)

def five_nearest_weather_stations(latitude, longitude):
    query_coordinates = np.array([[latitude, longitude]])
    distances, indices = neighbors_model.kneighbors(query_coordinates)
    nearest_stations_info = stations.iloc[indices[0]].copy() 
    nearest_stations_info['DISTANCE(°)'] = distances[0]
    
    # Calculation of actual distance (km)
    nearest_stations_info['DISTANCE(KM)'] = nearest_stations_info.apply(
        lambda row: haversine(longitude, latitude, row['LONGITUDE'], row['LATITUDE']), 
        axis=1
    )

    return nearest_stations_info

In [42]:
five_nearest_weather_stations(40,-86)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,STATION,LATITUDE,LONGITUDE,DISTANCE(°),DISTANCE(KM)
671,USW00093819,39.72515,-86.28160,0.393499,38.880677
839,USW00053866,39.58545,-85.79982,0.460352,49.166428
336,USC00120784,39.17399,-86.52076,0.976465,102.114701
0,USW00014835,40.41236,-86.94739,1.033242,92.602652
503,USW00014827,40.97248,-85.20636,1.255222,127.270106


In [43]:
# Simple average
def average_values_for_nearest_stations(latitude, longitude):
    nearest_stations_info = five_nearest_weather_stations(latitude, longitude)
    nearest_station_ids = nearest_stations_info['STATION'].tolist()
    filtered_data = data_cleaned[data_cleaned['STATION'].isin(nearest_station_ids)]
    average_values = filtered_data.groupby(['YEAR', 'MONTH'])[['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']].mean().reset_index()
    return average_values

In [44]:
average_values_for_nearest_stations(40,-86)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,YEAR,MONTH,AWND,PRCP,TAVG,TMAX,TMIN
0,2010,1,9.300,1.2120,24.20,30.46,17.96
1,2010,2,8.540,1.1600,25.72,32.52,18.90
2,2010,3,7.880,3.1220,44.62,54.90,34.40
3,2010,4,9.040,3.4380,58.36,70.30,46.42
4,2010,5,8.120,5.8400,65.10,75.02,55.18
...,...,...,...,...,...,...,...
163,2023,8,6.700,3.4600,72.58,82.46,62.66
164,2023,9,6.060,1.3260,68.22,79.92,56.52
165,2023,10,7.920,2.7700,56.78,66.88,46.68
166,2023,11,8.320,0.7900,43.96,55.02,32.88


## Predict precipitation (Random Forest)

In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [62]:
average_values_df = average_values_for_nearest_stations(40,-86)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [59]:
# Only use year and month as input and precipitation as output
X = average_values_df[['YEAR', 'MONTH']]
y = average_values_df['PRCP'] 
X_train = X[X['YEAR'] < 2023]
X_test = X[X['YEAR'] == 2023]
y_train = y[X['YEAR'] < 2023]
y_test = y[X['YEAR'] == 2023]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse}')

Test MSE: 1.4712721945854144


In [68]:
average_values_df = average_values_for_nearest_stations(40,-86)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [69]:
#Creating data with a 6-month lag
for var in ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']:
    average_values_df[f'{var}_lag12'] = average_values_df[var].shift(12)

average_values_df.dropna(inplace=True)
average_values_df

,YEAR,MONTH,AWND,PRCP,TAVG,TMAX,TMIN,AWND_lag12,PRCP_lag12,TAVG_lag12,TMAX_lag12,TMIN_lag12
12,2011,1,8.440,1.5920,23.22,30.40,16.10,9.30,1.212,24.20,30.46,17.96
13,2011,2,9.600,4.2220,31.70,40.02,23.40,8.54,1.160,25.72,32.52,18.90
14,2011,3,9.680,3.4560,42.30,51.48,33.14,7.88,3.122,44.62,54.90,34.40
15,2011,4,10.980,8.6460,54.40,64.94,43.88,9.04,3.438,58.36,70.30,46.42
16,2011,5,8.000,5.8020,63.32,72.82,53.82,8.12,5.840,65.10,75.02,55.18
...,...,...,...,...,...,...,...,...,...,...,...,...
163,2023,8,6.700,3.4600,72.58,82.46,62.66,6.42,3.494,73.90,84.32,63.46
164,2023,9,6.060,1.3260,68.22,79.92,56.52,6.62,2.208,66.82,77.60,56.02
165,2023,10,7.920,2.7700,56.78,66.88,46.68,8.44,1.376,53.86,66.30,41.44
166,2023,11,8.320,0.7900,43.96,55.02,32.88,8.98,1.382,44.06,54.14,34.02


In [70]:
X = average_values_df[['YEAR', 'MONTH'] + [f'{var}_lag12' for var in ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']]]
y = average_values_df['PRCP']
X_train = X[average_values_df['YEAR'] < 2023]
X_test = X[average_values_df['YEAR'] == 2023]
y_train = y[average_values_df['YEAR'] < 2023]
y_test = y[average_values_df['YEAR'] == 2023]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse}')

Test MSE: 2.542705880404165


In [74]:
average_values_df = average_values_for_nearest_stations(40,-90)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [75]:
# Only use year and month as input and precipitation as output
X = average_values_df[['YEAR', 'MONTH']]
y = average_values_df['PRCP'] 
X_train = X[X['YEAR'] < 2023]
X_test = X[X['YEAR'] == 2023]
y_train = y[X['YEAR'] < 2023]
y_test = y[X['YEAR'] == 2023]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse}')

Test MSE: 1.5343856393020807


In [72]:
#Creating data with a 6-month lag
for var in ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']:
    average_values_df[f'{var}_lag12'] = average_values_df[var].shift(12)

average_values_df.dropna(inplace=True)
average_values_df

,YEAR,MONTH,AWND,PRCP,TAVG,TMAX,TMIN,AWND_lag12,PRCP_lag12,TAVG_lag12,TMAX_lag12,TMIN_lag12
12,2011,1,8.080,0.766000,30.760,30.380,22.960,8.480,1.3320,30.760,29.620,23.220
13,2011,2,9.160,3.048000,36.220,38.240,27.780,7.740,1.2120,34.060,34.220,26.620
14,2011,3,8.760,3.164000,44.340,49.480,35.380,8.220,2.4660,46.720,53.320,37.120
15,2011,4,9.920,7.178000,54.200,62.140,44.500,9.160,2.9800,58.020,70.060,46.720
16,2011,5,8.900,5.494000,60.720,71.660,50.960,8.000,5.3180,63.260,74.160,53.440
...,...,...,...,...,...,...,...,...,...,...,...,...
164,2023,9,6.325,1.977500,68.475,78.900,57.975,6.900,2.0225,67.125,77.900,56.300
165,2023,10,8.100,2.852500,56.250,66.000,46.500,8.200,1.3675,53.425,65.900,40.975
166,2023,11,7.900,0.732500,42.425,52.950,31.925,9.100,1.5725,42.475,52.700,32.325
167,2023,12,8.200,2.230000,38.975,45.825,32.100,9.275,2.1550,29.575,37.325,21.825


In [73]:
X = average_values_df[['YEAR', 'MONTH'] + [f'{var}_lag12' for var in ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']]]
y = average_values_df['PRCP']
X_train = X[average_values_df['YEAR'] < 2023]
X_test = X[average_values_df['YEAR'] == 2023]
y_train = y[average_values_df['YEAR'] < 2023]
y_test = y[average_values_df['YEAR'] == 2023]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse}')

Test MSE: 2.758363465447916


In [86]:
average_values_df = average_values_for_nearest_stations(45,-70)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [79]:
five_nearest_weather_stations(45,-70)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,STATION,LATITUDE,LONGITUDE,DISTANCE(°),DISTANCE(KM)
2284,USC00174927,44.79760,-69.88900,0.230839,24.144410
1923,USW00094626,45.46310,-69.55526,0.642071,62.165492
1802,USW00014606,44.79791,-68.81852,1.198639,95.733237
2044,USW00054772,43.98992,-70.95006,1.386678,135.250375
2165,USC00174193,43.36050,-70.46970,1.705455,186.111056


In [87]:
# Only use year and month as input and precipitation as output
X = average_values_df[['YEAR', 'MONTH']]
y = average_values_df['PRCP'] 
X_train = X[X['YEAR'] < 2023]
X_test = X[X['YEAR'] == 2023]
y_train = y[X['YEAR'] < 2023]
y_test = y[X['YEAR'] == 2023]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse}')

Test MSE: 4.776500042866662


In [88]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)

print(f'XGB Test MSE: {mse_xgb}')

XGB Test MSE: 5.438861296252566


In [81]:
#Creating data with a 6-month lag
for var in ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']:
    average_values_df[f'{var}_lag12'] = average_values_df[var].shift(12)

average_values_df.dropna(inplace=True)
average_values_df

,YEAR,MONTH,AWND,PRCP,TAVG,TMAX,TMIN,AWND_lag12,PRCP_lag12,TAVG_lag12,TMAX_lag12,TMIN_lag12
12,2015,1,7.060,3.6120,14.480,25.88,3.100,6.84,3.450,16.70,26.52,6.90
13,2015,2,7.200,1.9320,6.900,19.02,-5.260,6.56,3.152,17.90,28.92,6.90
14,2015,3,7.860,1.7900,23.700,35.02,12.440,7.64,4.250,21.18,32.54,9.84
15,2015,4,7.420,3.3260,39.900,50.38,29.420,7.68,3.090,40.68,51.26,30.12
16,2015,5,6.820,1.8180,57.100,69.60,44.580,6.66,4.138,53.26,63.66,42.90
...,...,...,...,...,...,...,...,...,...,...,...,...
116,2023,9,5.700,4.1280,62.960,72.66,53.220,6.26,5.178,58.52,68.32,48.76
117,2023,10,6.120,4.7220,53.020,61.88,44.140,5.80,6.266,49.90,61.44,38.38
118,2023,11,6.760,2.7720,33.840,43.20,24.460,7.24,4.578,40.10,50.20,29.96
119,2023,12,6.200,6.9950,37.025,45.40,28.525,7.74,4.456,29.04,36.98,21.10


In [82]:
X = average_values_df[['YEAR', 'MONTH'] + [f'{var}_lag12' for var in ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN']]]
y = average_values_df['PRCP']
X_train = X[average_values_df['YEAR'] < 2023]
X_test = X[average_values_df['YEAR'] == 2023]
y_train = y[average_values_df['YEAR'] < 2023]
y_test = y[average_values_df['YEAR'] == 2023]
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse}')

Test MSE: 4.132169443566669


In [84]:
pip install xgboost

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 953.3 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [85]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)

print(f'XGB Test MSE: {mse_xgb}')

XGB Test MSE: 5.750323787561754
